## 0)import libraries 

In [1]:
"""
Thesis_PhD_Qianqian Predicted_fluxes_glboal_stripes
date: 24-Feb-2023
author: Qianqian
Contact: q.han@utwente.nl
-------------------------------------
Description: 
"""
# libraries
import os
import joblib
from osgeo import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from rasterio.warp import reproject, Resampling
import xarray as xr
import pickle
from PyStemmusScope import variable_conversion as vc
from rasterio.warp import reproject, Resampling
from sklearn.preprocessing import OneHotEncoder
import time
import re
import glob
import rioxarray
import dask.array
from dask.distributed import Client, progress

In [2]:
workingPath = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/"

In [3]:
# function for getting directory of input data
def get_directories_with_number_and_ending(directory_path, ending):
    directories = []
    if ending:
        pattern = re.compile(r'^\d+.*{}$'.format(re.escape(ending)))  # Match directory names with number at the beginning and specific ending
    else:
        pattern = re.compile(r'^\d+$')  # Match directory names with only numbers

    for entry in os.scandir(directory_path):
        if entry.is_dir():
            directory_name = entry.name
            if pattern.match(directory_name):
                directories.append(directory_name)

    return directories

## 3) read data

In [4]:
### define the path of input data, set the reference file for spatial resample
inputData = workingPath+"1input_data/"
# get all the filefolders named as year
year_list = get_directories_with_number_and_ending(workingPath+"1input_data/", "global")
# the input data from year[0] to year[..], based on the process id in sbatch script
year = year_list[4-1]
print(year)

2015global


In [5]:
import geopandas as gpd
from shapely.geometry import box

shapefile_path = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/Emulator/input_data/EuropeBoundary.shp"
gdf = gpd.read_file(shapefile_path)
bbox = gdf.total_bounds
bbox

array([-31.28903052,  34.93055094,  68.93136141,  81.85192337])

In [6]:
### 0) read era5land data
# select Europe data of ERA5Land and save it as netcdf
all1 = xr.open_mfdataset(inputData+year+"/era5land/era5-land*.nc", chunks='auto')
# all1 = xr.open_mfdataset(inputData+year+"/era5land/era5-land*.nc")
lon_name = 'longitude'
all1['longitude_adjusted'] = xr.where(
    all1[lon_name] > 180,
    all1[lon_name] - 360,
    all1[lon_name])
all1 = (
    all1
    .swap_dims({lon_name: 'longitude_adjusted'})
    .sel(**{'longitude_adjusted': sorted(all1.longitude_adjusted)})
    .drop(lon_name))
all1 = all1.rename({'longitude_adjusted': lon_name})
all1 = all1.sel(
    latitude=slice(bbox[3], bbox[1]),  
    longitude=slice(bbox[0], bbox[2])  
)

In [ ]:
all1.to_netcdf('/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2015global/era5land/era5land2015_10km3.nc',format='NETCDF4')